In [1]:
import json
import math

import numpy as np

import os
os.getcwd()
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True' # sonst raucht der Kernel auf dem Macbook Pro ab
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'


import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten

Using TensorFlow backend.


In [1]:
# Zeichnet Boards 
 XXXXXXXXXXXXXXXXXXXXXXX### XXX TBC: TEST AUF FEHLER EINBAUEN

def board_to_string (X,y, width = 3):
    output = "\n\nX = Stein von Spieler am Zug, 0 = Stein von anderem Spieler, * = Gute Züge\n\›"
    layer_char = ["X","0","A","B","C","D","E"] # eigentlich kommt nur X (spieler am zug), 0 (spieler am zug) und * als empfohlener Zug vor 
    for i, data_elem in enumerate (X):
        output += ("\ndata_elem: " + str(i) + "\n\n")
        for j in range (pixels):
            t_set = False
            for k, layer in enumerate (data_elem):
                if layer [j] == 1:
                    output += layer_char [k]
                    t_set = True
            if y [i][j] == 1:
                output += "*"
                t_set = True                    
            if not t_set:
                output += "."                
            if (j + 1) % width == 0:
                output += "\n"      
        output += "\n"                  
    return output


# nimmt eine stellung in der form array[0,1,2,0,0,2,1,0,0], z.B. .
# Stellung 0:
#    xx.
#    .00
#    x..
# und gibt ein array [8,9] aus: Die Stellung selbst, drei Drehungen (90°,180°,270°) und die 4 Spiegelungen von Stellung 0-3 an der X oder Y Achse (egal)
# Stellung 1:
#   x.x 
#   .0x
#   .0.
#   
# Spiegelung
#   .0.
#   .0x
#   x.x
#
# Die Drehungen und Spiegelungen sind topologisch aus der Sicht des Spiels identisch, wenn die Folgezüge, dann auch jeweils gedreht und gespiegelt werden
#
#

def transform(input_array):
    corners = [[1,3,9,7], [7,9,3,1]]
    edges = [[2,6,8,4], [8,6,2,4]]
    output_arrays = []
    for variant in range(2):
        for turn in range(4):
            new_array = [0,0,0,0,0,0,0,0,0]
            for position in range(1,10):
                for i in range(4):
                    if position == corners[variant][i]:
                        new_array[corners[0][(i+turn)%4]-1] = input_array[position-1]
                    elif position == edges[variant][i]:
                        new_array[edges[0][(i+turn)%4]-1] = input_array[position-1]
                    elif position == 5:
                        new_array[4] = input_array[position-1]
            output_arrays.append(new_array)
    return output_arrays

In [2]:
def read_training_data (filename): # returns JSON object as a dictionary
    datafile = open(filename)
    dataset = json.load(datafile)   
    return dataset

def X_y_training_data (dataset, layers=2, pixels=5*5):   # returns X and y out of JSON dataset
    datasize = len(dataset)
    X = np.zeros([datasize, layers, pixels])   # e.g. 30 data samples with 2 (p)layers and a board of 5 x 5 
    y = np.zeros([datasize, pixels])           # maximum possible moves = pixels
    for i, data_elem in enumerate(dataset): # data_elem = 1 learning sample 
        # print ("*** data_elem", i,"-","\n", data_elem)        
        for j, fieldvalue in enumerate (data_elem["board"]):
            # print ("j:",j, " fieldvalue:",fieldvalue)
            # The layer of the drawing player
            if fieldvalue == 1:
                X[i,0,j] = 1.0
            elif fieldvalue == 2:
                X[i,1,j] = 1.0
            # if fieldvalue != 0:   
            #     X[i,fieldvalue-1,j] = 1.0
        for j, rated_move in enumerate (data_elem["rated_moves"]):
            y[i,j] = rated_move   
            # print ("j:",j, " ratedmove:",rated_move)
    return X, y  


def initialize_model (input_layers=2, pixels=25, neurons=90):
    model = Sequential()
    model.add(Dense(pixels, input_shape=(input_layers,pixels), activation='relu',name="input"))
    model.add(Flatten(name="flatten"))
    model.add(Dense(neurons, activation='relu', name="h1"))
    model.add(Dense(neurons, activation='relu', name="h2"))
    model.add(Dense(neurons, activation='relu', name="h3"))
    model.add(Dense(pixels, name="output"))
    # model.summary ()        
    return model

def train_model (model, X, y, epochs=50, batch_size=2):    
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mse'])

    # fit the keras model on the dataset
    model.fit (X, y, epochs=epochs, batch_size=batch_size, verbose=0)

    # evaluate the keras model
    mae, mse = model.evaluate(X, y)
    print('MSE train: %.4f' % (mse))
    # mae, mse = model.evaluate(X_test, y_test)
    # print('Mean squared error test: %.4f' % (mse))
    
    
def drawBoard(match):
    bu = "┌───┬───┬───┬───┬───┐"
    bm = "├───┼───┼───┼───┼───┤"
    bd = "└───┴───┴───┴───┴───┘"
    for row in range(11):
        for i, turn in enumerate(match):
            # select the boards to be shown           
            if i != 0:
                if row == 0:
                    print(bu, end="   ")
                elif row == 10:
                    print(bd, end="   ")
                elif row % 2 == 0:
                    print(bm, end="   ")
                else:
                    for j in range(5):
                        mark = str(turn["board"][int(j+5/2*(row-1))])
                        if mark == "0":
                            mark = " "
                        elif mark == "1":
                            mark = "X"
                        elif mark == "2":
                            mark = "O"
                        elif mark == "-1":
                            mark = "*"
                        print("│ " + mark + " ", end="")
                    print("│   ", end="")
        print("")

def drawBoardTight(match):    
    bu = "┌─┬─┬─┬─┬─┐"
    bm = "├─┼─┼─┼─┼─┤"
    bd = "└─┴─┴─┴─┴─┘"
    for row in range(11):
        for i, turn in enumerate(match):
            # select the boards to be shown           
            if i != 0:
                if row == 0:
                    print(bu, end=" ")
                elif row == 10:
                    print(bd, end=" ")
                elif row % 2 == 0:
                    print(bm, end=" ")
                else:
                    for j in range(5):
                        mark = str(turn["board"][int(j+5/2*(row-1))])
                        if mark == "0":
                            mark = " "
                        elif mark == "1":
                            mark = "X"
                        elif mark == "2":
                            mark = "O"
                        elif mark == "-1":
                            mark = "*"
                        print("│" + mark + "", end="")
                    print("│ ", end="")
        print("")        

def drawSimpBoard(predictions,roundto):      # draws predictions in a neat board 5*5
    for i, prediction in enumerate (predictions):
        output_prediction = prediction
        c = "{:" + str(roundto+3) + "." + str(roundto) +  "f}"
        if i % 5 == 0:
            print("\n" + " "*8, end="")
        print(c.format(output_prediction), end=" ")
    print("")
        
def add_to_learned (learned_data, board, rated_moves):   # adds to learned data, a board (5*5 pixel = current situation) and rated moves -3 / -2 disqualification, 0 neutral, 1 good, -1 bad 

    #print ("DIAGNOSE@@@@@@@@@@@@@@@LD", learned_data)
    #print ("DIAGNOSE@@@@@@@@@@@@@@@BOARD", board)
    #print ("DIAGNOSE@@@@@@@@@@@@@@@RM", rated_moves)

    index = next((index for (index, value) in enumerate(learned_data) if np.array_equal(value["board"], board)), None)        # falls board in learned_data enthalten ist, wird sein index bestimmt
    position = np.argmax (np.abs(rated_moves)) # np.nonzero(rated_moves)[0][0]    
    # an welcher stelle ist der neue zu lernende wert in rates moves 
    # print (index, position)
    count = np.abs(rated_moves) / np.amax(np.abs(rated_moves)) # np.insert(np.zeros(24),position,1)
    # count = np.zeros(25)
    # count[position] = 1
    # count array mit 1 an der stelle des ratings; sonst 0
        
    if index != None:
        learned_data[index]["rated_moves"][position] *= learned_data[index]["count_rm"][position]  # rated_moves sind nicht mehr gemittelt
        learned_data[index]["rated_moves"][position] += rated_moves[position]                      # rated_moves eines boards werden zu den schon existierenden dazuaddiert
        learned_data[index]["count_rm"]   [position] += 1 #count[position]                                         # count wird um einzelnen count ergänzt (an entsprechnden stelle += 1 gemacht)
        learned_data[index]["rated_moves"][position] /= learned_data[index]["count_rm"][position]  # rated_moves wird durch neues count geteilt, ist wieder gemittelt
    else:
        learned_data = np.append(learned_data, [{"board": board, "rated_moves": rated_moves, "count_rm": count}]) # falls board noch nicht existiert, wird es zu learned_data appended... geht das appenden so? (mit axis=0)
    return learned_data

ld_test_1 = np.array ([{"board":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"rated_moves":[0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0],"count_rm":[0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0]}]) 
board1 = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
ratmo1 = np.array([0,-3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
board2 = np.array([0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
ratmo2 = np.array([0,0,-3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
board3 = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0])
ratmo3 = np.array([0,0,0,-3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

ld_test_1 = add_to_learned (ld_test_1, board1, ratmo1)

# print (ld_test_1)

ld_test_2 = add_to_learned (ld_test_1, board2, ratmo2)

# print (ld_test_2)

ld_test_3 = add_to_learned (ld_test_2, board3, ratmo3)

# print (ld_test_3)


NameError: name 'np' is not defined

In [9]:
#
# TODO: VERBOSITY LEVEL INTEGRIEREN!
# 


n_size = 50 # Anzahl der großen Runden, die je aus einer Reihe von Spielen und einer Reihe von Trainingsläufen besteht
m_size = 500 # Anzahl der matches in einem Trainingslauf
           # l = das aktuelle Spiel  

players = [{},{}] # player[0]['model'] etc. ...

fields_outside = (0,1,2,3,4,5,9,10,14,15,19,20,21,22,23,24)
winning_sets = {(6,7,8),(11,12,13),(16,17,18),(6,11,16),(7,12,17),(8,13,18),(6,12,18),(8,12,16)}
treshold_count_to_use_data = 50 # how many count_rated_data must a board have, to use it
 
outside_rating = - 3
double_rating  = - 2 
good_move_rating = 5  # bad_move_rating = -good_move_rating
not_fatal_move = 1


learned_data = np.array ([{"board":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"rated_moves":[0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0],"count_rm":[0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0]}]) 
# print (learned_data)

for n in range (n_size): ## große runde, mehrere trainings, mehrere matches
    print ("\n\n\n\n\n—————————————————————————————————————————————————————\nRUN:",n,"\n") 
    # print ("—————————————————————————————————————————————————————\nRUN:",n,"\n") 
    # print ("—————————————————————————————————————————————————————\nRUN:",n,"\n") 
    # print ("—————————————————————————————————————————————————————\nRUN:",n,"\n") 
    
    # print ("—————————————————————————————————————————————————————\nRUN:",n,"\n") 
    # create model
    for i, player in enumerate (players):
        player ['model'] = initialize_model (input_layers=2, pixels=25, neurons=90)
    # players [0]['model'].summary ()
    # players [1]['model'].summary ()

    # fetch training data    
    if n == 0:
        dataset = read_training_data ('../01_data/Data_25F_01.json')
        X,y = X_y_training_data (dataset)
    else: 
        filtered_learned_data = []
        for element in learned_data:
            if np.sum(element["count_rm"]) > treshold_count_to_use_data: 
                filtered_learned_data.append(element)
        # print (filtered_learned_data)
        X,y = X_y_training_data (filtered_learned_data)
    
    # print (X)
    # print (y)
    if len (X) > 30:
        batch_size = 30
    else:
        batch_size = len (X)
    if len (X) > 0:
        for i, player in enumerate (players):
            # print ("\n\nmodel player:",i)
            train_model (player['model'], X, y, epochs=300, batch_size=batch_size)
    
    # Wir lassen die Predictions in den ersten runs etwas variieren, damit im Match mehrere Zugvarianten gelernt werden können
    variance = 1/(n+1) # alt.: e^(-n), muss definitv noch getweakt werden, momentan noch zu hoch
    print("# variance:", variance)
    mean = 0
    standard_deviation = math.sqrt(variance)
    
    for ma in range (m_size):
        print ("\n\n\n[ MATCH",ma,"]\n")
        board = np.zeros (25, dtype=int) # start_board is empty
        match = []
        
        # print (board)
        done = False
        for turn in range(9):   # while schleife?
            # board = match[-1]["board"]
            # print ("\nturn:", turn, "board:", board)
            X_board = np.zeros ((2,25))
            for e, element in enumerate (board):
                if element != 0:
                     X_board[element-1][e] = 1.0
 
            
            p = int (turn % 2) #int????   # wer ist dran, runde 0,2,4,6,8 => p0 1,3,5,7 => p1
            deviations = np.random.normal(mean, standard_deviation, 25)
            predictions = players [p]['model'].predict(np.array([X_board,]))  # Komma???
            chosen = np.argmax(predictions + deviations)
            
            # print (X_board)
            # print ("# player:", p)
            # print("# deviations:", np.around(deviations,3))
            print("\n    (" + str(turn) + ") raw_predict:")
            drawSimpBoard(predictions[0],3) # copy() ???
            print("\n        chosen:", chosen)
            # print("# alt_predict:", np.around(predictions + deviations,1))
            
            # schreiben in matches
            match.append({"board":np.copy(board),"move":chosen}) # Rocky: Copy!!!!
            # print ("match:", match)
            if board[chosen] == 0:
                board[chosen] = p+1      # next board with actual move, p0 = 1, p1 = 2, empty = 0 ### nach oben??? 
            else:
                board[chosen] = -1               
            # print ("board:", board)
            
            # Fertig? 4 Abbruchbedingungen:
                # 1) Mark außerhalb des Spielfelds
                # 2) Mark auf schon belegtes Feld
                # 3) 3 in einer Reihe
                # 4) 9. Zug (Feld voll)
            
            # Outside
            if chosen in fields_outside:
                rated_moves = np.zeros(25)
                rated_moves[chosen] = outside_rating
                learned_data = add_to_learned (learned_data, np.copy(match[-1]["board"]),np.copy(rated_moves)) #nur board??? wieso copy?          
                print ("\n# Outside! (field " + str(chosen) + ")\n")
                done = True
            
            # Double
            if board[chosen] == -1:
                rated_moves = np.zeros(25)
                rated_moves[chosen] = double_rating
                learned_data = add_to_learned (learned_data, np.copy(match[-1]["board"]),np.copy(rated_moves)) # "", ""
                print ("\n# Double! (field " + str(chosen) + ")\n")
                done = True
                
            # Victory
            for x in winning_sets:  # wir checken jede der 8 gewinnmöglichkeiten 6,7,8 - 11,12,13 - 16,17,18 ... ob sie im aktuellen board mit der zahl des players belegt ist 
                # print("x:",x)
                victory = True # wenn alle drei felder von player belegt sind, bleibt es true
                for y in x:
                    # print("p:", p)
                    # print("y:", y)
                    # print("board[y]:", board[y])
                    if board[y] != p+1:
                        victory = False
                    # print("victory1:", victory)
                # print("victory2:", victory)
                if victory:        # alle drei felder sind von player belegt 
                    for i, t in enumerate(match): # wir gehen ALLE turns in match durch (bis zu 9)
                        rated_moves = np.zeros(25) 
                        rated_moves[t["move"]] = (-1)**(i+p)*good_move_rating*(i+1)/len(match) # es setzt bei 5 zügen (bedeutet p = 0 hat gewonnen) für player0 auf good_move_rating * 0.2 für den ersten zug, für p1 => -gmr * 0.4 für den zweiten zug und für p0 = gmr * 0.6 für den dritten, etc.
                        print("rated_moves:", np.around(rated_moves,3))
                        learned_data = add_to_learned (learned_data, np.copy(t["board"]),np.copy(rated_moves))
                    print("\n# Gewinn! (field " + str(chosen) + ")", x, "\n")
                    done = True

            # if !done Normaler non_fatal_move Zug ohne sofort zu scheitern oder zu gewinnen => direkte Belohnung, z.B. +0.5
            
                    
            if done:
                break
            
            
            # next move
            
        # Das ist doppelt, s.o.
        # Wo anders hin? Wir müssen auch 2 übereinander abbilden
        # board[chosen] = p+1 # next board with actual move, p0 = 1, p2 = 2, empty = 0 ??????????????????????
        match.append ({"board":np.copy(board),"move":-1}) # ???
        #print("match:", match)
        drawBoardTight(match)
        # print ("\n\nmatch:", match)                  
        # print ("\n\nlearned_data", learned_data)      
        # print ("\n\ndataset", dataset)     
        # print (learned_data[1]["rated_moves"][22])
        # print (learned_data[1]["rated_moves"][3])
       
    # print (dataset[0]["rated_moves"][21])

print ("\n\n\n learned_data", learned_data)
print ("\n\n\n filtered_learned_data", filtered_learned_data)






—————————————————————————————————————————————————————
RUN: 0 

12/12 [==============================] - 0s 11ms/step
MSE train: 0.0771


KeyboardInterrupt: 

In [26]:

filtered_learned_data = []

for element in learned_data:
    if np.sum(element["count_rm"]) > 50:
        filtered_learned_data.append(element)

print (filtered_learned_data)

[{'board': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'rated_moves': [-3.0, -3.0, -3.0, -3.0, -3.0, -3.0, 1, 1, 1, -3.0, -3.0, 1, 1, 1, -3.0, -3.0, 1, 1, 1, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0], 'count_rm': [198, 189, 192, 222, 190, 174, 1, 1, 1, 179, 206, 1, 1, 1, 192, 172, 1, 1, 1, 179, 190, 196, 219, 163, 174]}, {'board': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0]), 'rated_moves': array([-3., -3., -3., -3., -3., -3.,  0.,  0.,  0., -3., -3.,  0.,  0.,
        0., -3., -3., -2.,  0.,  0., -3., -3., -3., -3., -3., -3.]), 'count_rm': array([ 4., 10.,  6., 10.,  5., 14.,  0.,  0.,  0.,  9.,  8.,  0.,  0.,
        0.,  6.,  8., 17.,  0.,  0., 11.,  3., 16., 12.,  7.,  7.])}, {'board': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0]), 'rated_moves': array([-3., -3., -3., -3., -3., -3.,  0.,  0.,  0., -3., -3.,  0.,  0.,
       -2., -3., -3.,  0.,  0.,  0., -3., -3., -3.,

In [165]:
learned_data = np.array ([{"board":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"rated_moves":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2],"counter":1},{"board":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],"rated_moves":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,7],"counter":1},{"board":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],"rated_moves":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,11],"counter":1}])

el1 = {"board":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"rated_moves":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,6],"counter":1}
el2 = {"board":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],"rated_moves":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5],"counter":1}

# learned_data = np.append (learned_data, ela)
# learned_data = np.append (learned_data, elb)
print (learned_data)

# next(item for item in learned_data if item["board"] == el1["board"])
# next(item for item in learned_data if item["board"] == el2["board"])

index = next((index for (index, value) in enumerate(learned_data) if value["board"] == el1["board"]), None)
print ("\nindex:", index)

learned_data[0]["rated_moves"] = np.add (learned_data[0]["rated_moves"], el1["rated_moves"])
learned_data[0]["counter"] += 1
learned_data[1]["rated_moves"] = np.add (learned_data[1]["rated_moves"], el2["rated_moves"])
learned_data[1]["counter"] += 1

#### COUNTER ALS ARRAY!
                                         
print ("\nlearned_data:",learned_data)

average = np.multiply (learned_data[0]["rated_moves"], [1.0 / learned_data[0]["counter"]])

print ("\n\n",average)
# 

[{'board': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'rated_moves': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2], 'counter': 1}
 {'board': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 'rated_moves': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 7], 'counter': 1}
 {'board': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 'rated_moves': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 11], 'counter': 1}]

index: 0

learned_data: [{'board': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'rated_moves': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 2, 8]), 'counter': 2}
 {'board': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 'rated_moves': array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
    

In [ ]:
match = [{'board': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'move': 12},
         {'board': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'move': 7},
         {'board': [0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'move': 8},
         {'board': [0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'move': 16},
         {'board': [0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0], 'move': 13},
         {'board': [0 ,0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0], 'move': 11},
         {'board': [0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 2, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0], 'move': 18},
         {'board': [0 ,0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 2, 1, 1, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0], 'move': ""}]


drawBoardTight(match)

In [18]:
an_array = np.array([1, 1, 3,7,2])


max_index_col = np.argmax(an_array)
print (max_index_col)

3


In [10]:
an_array = np.empty((2, 2))

print(an_array)

[[-2.68156159e+154 -2.68679737e+154]
 [ 6.95116975e-310  2.78134232e-309]]


/Users/maximilianvogel/opt/anaconda3/envs/keras_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [19]:
position = 23
print(np.insert(np.zeros(24),position,1))
#print(np.concatenate((np.zeros(position),np.array([1]),np.zeros(24-position))))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0.]


In [9]:
rated_moves=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
np.nonzero(rated_moves)[0][0]

13

In [22]:
turn =5
print(turn % 2)

1


In [94]:
predictions = np.array([0.8,1.2,0.7,0.4,1.1,-1.4,1.9,0.5,1.3])
deviations = np.around(np.random.normal(0, 1, 9),1)
print(predictions)
print(deviations)
print(deviations+predictions)

[ 0.8  1.2  0.7  0.4  1.1 -1.4  1.9  0.5  1.3]
[ 1.5 -1.7 -1.3 -1.3  0.7 -0.4 -0.9  1.  -0.4]
[ 2.3 -0.5 -0.6 -0.9  1.8 -1.8  1.   1.5  0.9]


In [40]:
def drawSimpBoard(predictions):
    for i, prediction in enumerate (predictions):
        c = " "
        if i % 5 == 0:
            print("")
        if prediction < 0:
            c = ""
        print(c + str(prediction), end=" ")
    print("")

drawSimpBoard([-0.7, -0.9, -1.0, 0.0, -1.0, -1.1, -0.1,  0.0, 0.2, -0.8, -1.1,  0.0, -0.6, -0.4,
  -0.9, -0.8, -0.4, -0.1, -0.4, -0.8, -0.9, -0.9, 0.5, -1.1, -0.8])


-0.7 -0.9 -1.0  0.0 -1.0 
-1.1 -0.1  0.0  0.2 -0.8 
-1.1  0.0 -0.6 -0.4 -0.9 
-0.8 -0.4 -0.1 -0.4 -0.8 
-0.9 -0.9  0.5 -1.1 -0.8 
lol


In [3]:
datafile = open('../01_data/Data_03.json',)

layers = 2
pixels = 3 * 3

# returns JSON object as 
# a dictionary
dataset = json.load(datafile)


# print (dataset)
# print (dataset[1]["field"][4])
# print (dataset[2]["good_moves"][2])

# cleanly suck it up
datasize = len (dataset)
X = np.zeros([datasize, layers, pixels]) # e.g. 30 data samples with 2 (p)layers and a board of 3 x 3 
y = np.zeros([datasize, pixels]) # maximum possible moves = pixels

for i, data_elem in enumerate (dataset): # data_elem = 1 learning sample 
    # print ("*** data_elem", i,"-", data_elem)
    
    # who's turn is it? Die Steine des ziehenden Spieler kommen in input layer 0 des nnetzes, die des anderen in 1
    t_num, t_counts = np.unique(data_elem["board"], return_counts=True) # gibt etwas wie [0,1,2] [6,2,1] zurück => 2 Steine von P1, 1 von P2 
    counts = dict(zip(t_num, t_counts))
    if not 1 in counts: counts[1] = 0
    if not 2 in counts: counts[2] = 0
    # print ("counts", counts)        
    if counts[1] > counts[2]: # wenn P1 einen stein mehr auf dem Feld hat als P2, ist P2 dran, sonst P1
        turn = 2
    else:         
        turn = 1
    # print ("turn",turn)
    
    for j, pixel in enumerate (data_elem["board"]):
        # print (j, pixel)
        # The layer of the drawing player
        if pixel == turn:
            X[i,0,j] = 1.0
        elif pixel > 0:
                X[i,1,j] = 1.0
    for j, good_move in enumerate (data_elem["good_moves"]):
        y[i,good_move-1] = 1.0
        
     
                
# print ("X:\n", X) 
# print ("...")
# print ("y:\n", y)
            




    

In [5]:
# enriched_with_all 1+7 transformations
X_enriched = np.zeros([datasize*8, layers, pixels]) # e.g. 30 data samples with 2 (p)layers and a board of 3 x 3 
y_enriched = np.zeros([datasize*8, pixels]) # maximum possible moves = pixels


for i, data_elem in enumerate (X): # data_elem = 1 learning sample 
        # print ("# i", i)
        for j, layer in enumerate (data_elem):
            # print ("## j", j)
            for k, transformation_layer in enumerate (transform (layer)):
                # print (k, transformation_layer)
                X_enriched [i*8+k,j] = transformation_layer
            ### Papusch, hier noch schalter rein, dass es nur jedes zweite mal aufgerufen wird
            if (j == 0):
                for k, transformation_layer in enumerate (transform (y [i])):
                    y_enriched [i*8+k] = transformation_layer
        

print (board_to_string (X_enriched, y_enriched))





X = Stein von Spieler am Zug, 0 = Stein von anderem Spieler, * = Gute Züge
\›
data_elem: 0

***
***
***


data_elem: 1

***
***
***


data_elem: 2

***
***
***


data_elem: 3

***
***
***


data_elem: 4

***
***
***


data_elem: 5

***
***
***


data_elem: 6

***
***
***


data_elem: 7

***
***
***


data_elem: 8

*.*
.0.
*.*


data_elem: 9

*.*
.0.
*.*


data_elem: 10

*.*
.0.
*.*


data_elem: 11

*.*
.0.
*.*


data_elem: 12

*.*
.0.
*.*


data_elem: 13

*.*
.0.
*.*


data_elem: 14

*.*
.0.
*.*


data_elem: 15

*.*
.0.
*.*


data_elem: 16

0**
*X*
***


data_elem: 17

**0
*X*
***


data_elem: 18

***
*X*
**0


data_elem: 19

***
*X*
0**


data_elem: 20

***
*X*
0**


data_elem: 21

0**
*X*
***


data_elem: 22

**0
*X*
***


data_elem: 23

***
*X*
**0


data_elem: 24

X0.
.0.
.*.


data_elem: 25

..X
*00
...


data_elem: 26

.*.
.0.
.0X


data_elem: 27

...
00*
X..


data_elem: 28

.*.
.0.
X0.


data_elem: 29

X..
00*
...


data_elem: 30

.0X
.0.
.*.


data_elem: 31

...
*00
..X


da

In [11]:

# bringt die X und y in eine zufällige (aber gleiche) Reihenfolge

shuffler = np.random.permutation(len(X_enriched))
X_enriched = X_enriched[shuffler]
y_enriched = y_enriched[shuffler]

print (board_to_string (X_enriched, y_enriched))




X = Stein von Spieler am Zug, 0 = Stein von anderem Spieler, * = Gute Züge
\›
data_elem: 0

..0
.0.
*X.


data_elem: 1

X.*
*00
.X0


data_elem: 2

X.0
*..
X.0


data_elem: 3

*.0
0XX
.0X


data_elem: 4

0**
X*0
X0X


data_elem: 5

..*
0X.
X0.


data_elem: 6

0.*
...
*.X


data_elem: 7

...
.*.
0..


data_elem: 8

0..
.*.
X.0


data_elem: 9

0.X
.0X
..*


data_elem: 10

0..
X*.
X0.


data_elem: 11

X.0
0X.
..*


data_elem: 12

..X
0.X
0.*


data_elem: 13

..0
0X.
X**


data_elem: 14

*0X
*X0
0**


data_elem: 15

*..
0.X
0..


data_elem: 16

..0
.0.
XX*


data_elem: 17

.0X
0X.
*..


data_elem: 18

...
X..
0*0


data_elem: 19

.0X
.X.
*0.


data_elem: 20

*..
.X.
00X


data_elem: 21

**0
.*X
*..


data_elem: 22

0XX
.X.
*00


data_elem: 23

*.*
.0.
*.*


data_elem: 24

0*0
X*.
0.X


data_elem: 25

*..
.0.
.X0


data_elem: 26

X0.
.0.
.*.


data_elem: 27

X..
*.0
X0.


data_elem: 28

X0X
.*0
0.X


data_elem: 29

X0.
0X.
..*


data_elem: 30

**X
*0*
***


data_elem: 31

0..
0X.
*..


da

In [12]:
splitter = 128; # number of test samples, always the last in the file


X_train = X_enriched [:datasize*8 - splitter]
y_train = y_enriched [:datasize*8 - splitter]

X_test = X_enriched [-splitter:]
y_test = y_enriched [-splitter:]


print (board_to_string (X_train, y_train))

print ("\n\nTESTDATA\n")
print (board_to_string (X_test, y_test))


print ("X_train:\n", X_train) 
print ("y_train:\n", y_train) 
# print ("###")
# print ("X_test:\n", X_test) 
# print ("y_test:\n", y_test) 
# print ("###")
# print ("###")



X = Stein von Spieler am Zug, 0 = Stein von anderem Spieler, * = Gute Züge
\›
data_elem: 0

..0
.0.
*X.


data_elem: 1

X.*
*00
.X0


data_elem: 2

X.0
*..
X.0


data_elem: 3

*.0
0XX
.0X


data_elem: 4

0**
X*0
X0X


data_elem: 5

..*
0X.
X0.


data_elem: 6

0.*
...
*.X


data_elem: 7

...
.*.
0..


data_elem: 8

0..
.*.
X.0


data_elem: 9

0.X
.0X
..*


data_elem: 10

0..
X*.
X0.


data_elem: 11

X.0
0X.
..*


data_elem: 12

..X
0.X
0.*


data_elem: 13

..0
0X.
X**


data_elem: 14

*0X
*X0
0**


data_elem: 15

*..
0.X
0..


data_elem: 16

..0
.0.
XX*


data_elem: 17

.0X
0X.
*..


data_elem: 18

...
X..
0*0


data_elem: 19

.0X
.X.
*0.


data_elem: 20

*..
.X.
00X


data_elem: 21

**0
.*X
*..


data_elem: 22

0XX
.X.
*00


data_elem: 23

*.*
.0.
*.*


data_elem: 24

0*0
X*.
0.X


data_elem: 25

*..
.0.
.X0


data_elem: 26

X0.
.0.
.*.


data_elem: 27

X..
*.0
X0.


data_elem: 28

X0X
.*0
0.X


data_elem: 29

X0.
0X.
..*


data_elem: 30

**X
*0*
***


data_elem: 31

0..
0X.
*..


da

In [14]:
# Anmerkung Rocky: Mal die Anzahl der Neuronen variieren

model = Sequential()
model.add(Dense(pixels, input_shape=(layers,pixels), activation='relu',name="input"))
model.add(Flatten(name="flatten"))
model.add(Dense(90, activation='relu', name="h1"))
model.add(Dense(90, activation='relu', name="h2"))
model.add(Dense(90, activation='relu', name="h3"))
# model.add(Dense(80, activation='relu', name="h3"))
# model.add(Dense(80, activation='relu', name="h4"))
#model.add(Dense(60, activation='relu', name="h4"))
#model.add(Dense(40, activation='relu', name="h4"))
#model.add(Dense(pixels*layers, activation='relu', name="h4"))
#model.add(Dense(pixels*layers, activation='relu', name="h5"))
model.add(Dense(pixels, name="output"))
model.summary ()

# compile the keras model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.1), loss='mean_absolute_error')
# model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy'])

model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mse'])

# fit the keras model on the dataset
model.fit (X_train, y_train, epochs=500, batch_size=30)

# evaluate the keras model
mae, mse = model.evaluate(X_train, y_train)
print('Mean squared error train: %.4f' % (mse))
mae, mse = model.evaluate(X_test, y_test)
print('Mean squared error test: %.4f' % (mse))

# make class predictions with the model
predictions_train = model.predict(X_train)
predictions_test = model.predict(X_test)


# print (len(y_train))

#print ("\n\n### TRAIN ###")
#for i, elem in enumerate (predictions_train):
#    print(i, np.around(predictions_train[i], 2), y_train[i])

print ("\n\n### TEST ###")  
for i, elem in enumerate (predictions_test):
    print(i, np.around(predictions_test[i], 2), y_test[i])
    

         



Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Dense)                (None, 2, 9)              90        
_________________________________________________________________
flatten (Flatten)            (None, 18)                0         
_________________________________________________________________
h1 (Dense)                   (None, 90)                1710      
_________________________________________________________________
h2 (Dense)                   (None, 90)                8190      
_________________________________________________________________
h3 (Dense)                   (None, 90)                8190      
_________________________________________________________________
output (Dense)               (None, 9)                 819       
Total params: 18,999
Trainable params: 18,999
Non-trainable params: 0
__________________________________________________

920/920 [==============================] - 0s 125us/step - loss: 0.0298 - mse: 0.0196
Epoch 155/500
920/920 [==============================] - 0s 119us/step - loss: 0.0292 - mse: 0.0195
Epoch 156/500
920/920 [==============================] - 0s 127us/step - loss: 0.0294 - mse: 0.0195
Epoch 157/500
920/920 [==============================] - 0s 126us/step - loss: 0.0294 - mse: 0.0195
Epoch 158/500
920/920 [==============================] - 0s 126us/step - loss: 0.0293 - mse: 0.0194
Epoch 159/500
920/920 [==============================] - 0s 132us/step - loss: 0.0286 - mse: 0.0193
Epoch 160/500
920/920 [==============================] - 0s 128us/step - loss: 0.0291 - mse: 0.0194
Epoch 161/500
920/920 [==============================] - 0s 130us/step - loss: 0.0285 - mse: 0.0192
Epoch 162/500
920/920 [==============================] - 0s 130us/step - loss: 0.0290 - mse: 0.0193
Epoch 163/500
920/920 [==============================] - 0s 147us/step - loss: 0.0289 - mse: 0.0193
Epoch 164/500


920/920 [==============================] - 0s 126us/step - loss: 0.0246 - mse: 0.0166
Epoch 237/500
920/920 [==============================] - 0s 127us/step - loss: 0.0245 - mse: 0.0165
Epoch 238/500
920/920 [==============================] - 0s 127us/step - loss: 0.0250 - mse: 0.0166
Epoch 239/500
920/920 [==============================] - 0s 128us/step - loss: 0.0251 - mse: 0.0166
Epoch 240/500
920/920 [==============================] - 0s 128us/step - loss: 0.0243 - mse: 0.0165
Epoch 241/500
920/920 [==============================] - 0s 131us/step - loss: 0.0247 - mse: 0.0165
Epoch 242/500
920/920 [==============================] - 0s 134us/step - loss: 0.0247 - mse: 0.0165
Epoch 243/500
920/920 [==============================] - 0s 147us/step - loss: 0.0253 - mse: 0.0166
Epoch 244/500
920/920 [==============================] - 0s 139us/step - loss: 0.0244 - mse: 0.0165
Epoch 245/500
920/920 [==============================] - 0s 141us/step - loss: 0.0241 - mse: 0.0164
Epoch 246/500


920/920 [==============================] - 0s 135us/step - loss: 0.0219 - mse: 0.0135
Epoch 319/500
920/920 [==============================] - 0s 133us/step - loss: 0.0219 - mse: 0.0132
Epoch 320/500
920/920 [==============================] - 0s 134us/step - loss: 0.0208 - mse: 0.0130
Epoch 321/500
920/920 [==============================] - 0s 149us/step - loss: 0.0210 - mse: 0.0128
Epoch 322/500
920/920 [==============================] - 0s 151us/step - loss: 0.0215 - mse: 0.0127
Epoch 323/500
920/920 [==============================] - 0s 143us/step - loss: 0.0209 - mse: 0.0124
Epoch 324/500
920/920 [==============================] - 0s 146us/step - loss: 0.0204 - mse: 0.0122
Epoch 325/500
920/920 [==============================] - 0s 144us/step - loss: 0.0204 - mse: 0.0120
Epoch 326/500
920/920 [==============================] - 0s 141us/step - loss: 0.0205 - mse: 0.0119
Epoch 327/500
920/920 [==============================] - 0s 142us/step - loss: 0.0202 - mse: 0.0117
Epoch 328/500


920/920 [==============================] - 0s 155us/step - loss: 0.0157 - mse: 0.0089
Epoch 401/500
920/920 [==============================] - 0s 143us/step - loss: 0.0157 - mse: 0.0089
Epoch 402/500
920/920 [==============================] - 0s 142us/step - loss: 0.0161 - mse: 0.0089
Epoch 403/500
920/920 [==============================] - 0s 135us/step - loss: 0.0157 - mse: 0.0089
Epoch 404/500
920/920 [==============================] - 0s 133us/step - loss: 0.0151 - mse: 0.0088
Epoch 405/500
920/920 [==============================] - 0s 135us/step - loss: 0.0156 - mse: 0.0089
Epoch 406/500
920/920 [==============================] - 0s 133us/step - loss: 0.0157 - mse: 0.0089
Epoch 407/500
920/920 [==============================] - 0s 128us/step - loss: 0.0168 - mse: 0.0090
Epoch 408/500
920/920 [==============================] - 0s 130us/step - loss: 0.0170 - mse: 0.0090
Epoch 409/500
920/920 [==============================] - 0s 131us/step - loss: 0.0160 - mse: 0.0089
Epoch 410/500


920/920 [==============================] - 0s 131us/step - loss: 0.0149 - mse: 0.0082
Epoch 483/500
920/920 [==============================] - 0s 136us/step - loss: 0.0147 - mse: 0.0081
Epoch 484/500
920/920 [==============================] - 0s 133us/step - loss: 0.0147 - mse: 0.0081
Epoch 485/500
920/920 [==============================] - 0s 130us/step - loss: 0.0148 - mse: 0.0081
Epoch 486/500
920/920 [==============================] - 0s 136us/step - loss: 0.0151 - mse: 0.0081
Epoch 487/500
920/920 [==============================] - 0s 135us/step - loss: 0.0143 - mse: 0.0081
Epoch 488/500
920/920 [==============================] - 0s 139us/step - loss: 0.0141 - mse: 0.0080
Epoch 489/500
920/920 [==============================] - 0s 148us/step - loss: 0.0139 - mse: 0.0080
Epoch 490/500
920/920 [==============================] - 0s 140us/step - loss: 0.0142 - mse: 0.0080
Epoch 491/500
920/920 [==============================] - 0s 131us/step - loss: 0.0146 - mse: 0.0081
Epoch 492/500


[[1. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 1. 0. 0.]]


In [36]:
# Erstes Array ist AI
a_1 = np.array([[[0., 0., 1., 1., 1., 0., 0., 1., 0.],[1., 1., 0., 0., 0., 0., 1., 0., 1.]],])

 
print (np.around (model.predict(a_1),2))

[[-0.   -0.   -0.    0.01  0.2   0.51 -0.    0.   -0.  ]]
